# Cities Analysis

This scripts correponds to the section `5.2 Comparison to observational data`

In [1]:
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
#import iris.plot as iplt
#import iris
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.colors as colors


from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import geopandas as gpd
from geopandas import GeoDataFrame as gdf
import pandas as pd

In [2]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]

# Global $\chi$

In [3]:
ds = xr.open_dataset("../nc_files/f09_mask.nc")
ds=ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))
ds=ds.reindex(lon=sorted(ds.lon))

In [4]:
vari_ls = ["chi_abd","chi_opt1","chi_hyg","bc_per","dst_per","ncl_per","pom_per","soa_per","so4_per"]
df = ds.sel(season=["JJA","DJF"])[vari_ls]\
.to_dataframe()
df.describe()

,chi_abd,chi_opt1,chi_hyg,bc_per,dst_per,ncl_per,pom_per,soa_per,so4_per
count,108527.000000,65909.000000,86307.000000,110592.000000,110592.000000,110592.000000,110592.000000,110592.000000,110592.000000
mean,66.852661,68.226440,56.054493,0.031354,0.048389,0.394192,0.158574,0.110165,0.257325
std,12.486314,10.186817,11.827573,0.041258,0.120722,0.337415,0.194789,0.183352,0.224112
min,12.616662,38.242340,20.234201,0.000056,0.000035,0.000107,0.000234,0.000058,0.005200
25%,57.980553,61.400860,47.827744,0.002969,0.002654,0.031905,0.014335,0.004439,0.091356
50%,68.727936,70.339058,53.637234,0.015392,0.007331,0.386254,0.077060,0.028177,0.202975
75%,76.288803,75.866364,62.690895,0.042815,0.031897,0.699597,0.222367,0.117908,0.337882
max,93.622704,94.437447,86.626915,0.284152,0.971027,0.983648,0.871376,0.983884,0.987179


In [5]:
gpd.datasets.available

['naturalearth_cities', 'naturalearth_lowres', 'nybb']

In [6]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
points= gdf(df.reset_index(), geometry=gpd.points_from_xy(df.reset_index().lon, df.reset_index().lat))
points.crs="EPSG:4326"
jps = gpd.sjoin(world, points, how='right', op='contains')
vari_ls_new = ["lat","lon","season"] + vari_ls + ["geometry"]
land = jps[~jps["index_left"].isnull()][vari_ls_new]
ocean = jps[jps["index_left"].isnull()][vari_ls_new]

## Define Cities

In [7]:
lat_array = np.array(list(set(land["lat"])))
lon_array = np.array(list(set(land["lon"])))

lat_dict = {"Pittsburgh": 40.4406, "Paris": 48.75, 
            "Sammaltunturi": 67.97333, 
            "Tokyo": 35.71}
lon_dict = {"Pittsburgh": -79.9959, "Paris": 2.36, 
            "Sammaltunturi": 24.11565, 
            "Tokyo": 139.76}

## Paris

In [8]:
Paris = land[(land["lat"]==find_nearest(lat_array, lat_dict["Paris"])) &
                         (land["lon"]==find_nearest(lon_array, lon_dict["Paris"]))]
display(Paris)

,lat,lon,season,chi_abd,chi_opt1,chi_hyg,bc_per,dst_per,ncl_per,pom_per,soa_per,so4_per,geometry
84965,48.534031,2.5,DJF,78.808830,67.704521,43.076012,0.096322,0.001442,0.125743,0.334059,0.333106,0.109328,POINT (2.50000 48.53403)
84964,48.534031,2.5,JJA,76.884262,66.178528,46.972713,0.056014,0.013732,0.039509,0.146800,0.496485,0.247460,POINT (2.50000 48.53403)


## Pittsburgh

In [9]:
Pittsburgh = land[(land["lat"]==find_nearest(lat_array, lat_dict["Pittsburgh"])) &
                         (land["lon"]==find_nearest(lon_array, lon_dict["Pittsburgh"]))]
display(Pittsburgh)
print("Mean:")
display(Pittsburgh.mean())

,lat,lon,season,chi_abd,chi_opt1,chi_hyg,bc_per,dst_per,ncl_per,pom_per,soa_per,so4_per,geometry
79649,40.052356,-80.0,DJF,77.698456,71.386299,45.394585,0.070552,0.000417,0.005213,0.294415,0.300415,0.328988,POINT (-80.00000 40.05236)
79648,40.052356,-80.0,JJA,78.213615,73.802887,54.655926,0.022439,0.001921,0.004341,0.109288,0.650958,0.211053,POINT (-80.00000 40.05236)


Mean:


lat         40.052356
lon        -80.000000
chi_abd     77.956039
chi_opt1    72.594589
chi_hyg     50.025253
bc_per       0.046496
dst_per      0.001169
ncl_per      0.004777
pom_per      0.201851
soa_per      0.475686
so4_per      0.270020
dtype: float64

## Finnish Arctic

In [10]:
Sammaltunturi = land[(land["lat"]==find_nearest(lat_array, lat_dict["Sammaltunturi"])) &
                         (land["lon"]==find_nearest(lon_array, lon_dict["Sammaltunturi"]))]
display(Sammaltunturi)
print("Mean:")
display(Sammaltunturi.mean())

,lat,lon,season,chi_abd,chi_opt1,chi_hyg,bc_per,dst_per,ncl_per,pom_per,soa_per,so4_per,geometry
97094,68.324607,23.75,JJA,81.342209,67.991493,49.851013,0.014588,0.002060,0.040986,0.117941,0.586534,0.237890,POINT (23.75000 68.32461)
97095,68.324607,23.75,DJF,72.503067,72.232559,55.608032,0.092398,0.011221,0.088400,0.385776,0.058759,0.363446,POINT (23.75000 68.32461)


Mean:


lat         68.324607
lon         23.750000
chi_abd     76.922638
chi_opt1    70.112030
chi_hyg     52.729523
bc_per       0.053493
dst_per      0.006641
ncl_per      0.064693
pom_per      0.251859
soa_per      0.322646
so4_per      0.300668
dtype: float64

## Tokyo 

In [11]:
Tokyo = land[(land["lat"]==find_nearest(lat_array, lat_dict["Tokyo"])) &
                         (land["lon"]==find_nearest(lon_array, lon_dict["Tokyo"]))]
display(Tokyo)

,lat,lon,season,chi_abd,chi_opt1,chi_hyg,bc_per,dst_per,ncl_per,pom_per,soa_per,so4_per,geometry
77120,35.340314,140.0,JJA,66.817795,60.615528,48.548016,0.065097,0.001559,0.039001,0.099606,0.354442,0.440295,POINT (140.00000 35.34031)
77121,35.340314,140.0,DJF,72.625435,66.294556,50.750072,0.123089,0.012829,0.032410,0.222414,0.187073,0.422185,POINT (140.00000 35.34031)
